# HDL targets

A functional description of a hardware design in emulated MyHDL or MyIRL generator constructs is normally output to VHDL or Verilog for transfer to a synthesis tool. The IRL kernel by default does that in a hierarchy-preserving way such that each `@component` or `@block` with a well defined interface results in a separate HDL component or module.

The IRL kernel generators impose some limitations on code elements, as in:

* Not allowing procedural loops in synthesizeable HDL transfer
* Not allowing certain operations with the default signal class

Therefore, when beginning a new design based on the next generation structural concepts, it helps to stick to the minimum functionality subset and extend later, when necessary.

As common examination subject, we create a minimal unit using simulation specific commands, i.e. it is used as a test bench in particular.

In [1]:
from myirl.emulation.myhdl import *
from myirl.simulation import *

@block
def unit():
    a = Signal(intbv(0xaa)[8:])
    a.init = True
    q = Signal(bool())

    @instance
    def stim():
        q.next = False
        if a[0] == True and a[1] == False and a[7] == False:  # True boolean evaluation
            q.next = True
            
        yield delay(1)
        assert q == False
    
        print("DONE")
        raise StopSimulation
    
    return instances()

## VHDL

VHDL output is the default verification transfer language for most unit tests. A `@block` is simply translated as follows:

To create the HDL code from `unit()`, we run:

In [2]:
u = unit()
files = u.elab(targets.VHDL)
files

 Writing 'unit' to file /tmp/myirl_unit_sj2qv384/unit.vhdl 


['/tmp/myirl_unit_sj2qv384/unit.vhdl']

Output the resulting file:

In [3]:
!cat {files[0]}

-- File generated from source:
--     /tmp/ipykernel_59573/2775767039.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity unit is
end entity unit;

architecture myhdl_emulation of unit is
    -- Local type declarations
    -- Signal declarations
    signal q : std_ulogic;
    signal a : unsigned(7 downto 0) := x"aa";
begin
    
stim:
    process
    begin
        q <= '0';
        if (((a(0) = '1') and (a(1) = '0')) and (a(7) = '0')) then
            q <= '1';
        end if;
        wait for 1 ns;
        assert (q = '0')
            report "Failed in /tmp/ipykernel_59573/2775767039.py:unit():17" severity failure;
        print("DONE");
        std.env.stop;
        wait;
    end process;
end architecture myhdl_emulation;



## Verilog

Verilog support is rudimentary and not complete for a number of wire types and class interfaces.
It was done as a proof of concept only and for now covers mainly:

* Simple arithmetics (`intbv` compatibility)
* Test bench generation

In [4]:
u = unit()
files = u.elab(targets.Verilog)
files

 Module unit: Existing instance unit, rename to unit_1 
 Writing 'unit_1' to file /tmp/myirl_unit_9hianvtd/unit_1.v 


['/tmp/myirl_unit_9hianvtd/unit_1.v']

Likewise, dump the Verilog output:

In [5]:
!cat {files[0]}

// File generated from source:
//     /tmp/ipykernel_59573/2775767039.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture myhdl_emulation

module unit_1
    ();
    // Local type declarations
    // Signal declarations
    reg  q;
    reg [7:0] a;
    initial a = 8'haa;
    
    initial begin : STIM
        q <= 1'b0;
        if ((((a[0] == 1'b1) & (a[1] == 1'b0)) & (a[7] == 1'b0))) begin
            q <= 1'b1;
        end
        #1.000000;
        `assert((q == 1'b0), "Failed in /tmp/ipykernel_59573/2775767039.py:unit():17") 
        $write("%s ", "DONE");
        $write("\n");
        $display("Stop Simulation");
        $finish;
    end
endmodule // unit_1


## Yosys / RTLIL

The API layer towards the libpyosys module was migrated from the `jupyosys` project to the IRL kernel.
This target directly creates RTL elements from the python HDL description.
Due to the yosys architecture, yosys interfacing may cause spurious aborts when certain commands are run. It is thus regarded unstable until yosys is made exception-safe for scripting APIs.

For more details, see [RTLIL target](target_rtlil.ipynb).

## Simulators

For simulation, the so far supported simulator back ends are:

* Icarus Verilog
* GHDL
* Yosys CXXRTL

The CXXRTL simulator digests synthesizeable hardware descriptions only, i.e. can not handle delay elements intrinsically. All simulation drivers are running natively in Python and co-simulate with a compiled CXXRTL module.

To run the above unit with one of the above HDL simulators, we finally call:

In [6]:
from myirl.test.common_test import Simulator

def test(uut, param = (), debug = False):
    inst = uut(*param)
    vhdl = targets.vhdl.VHDL()
    verilog = targets.verilog.Verilog()
    for tgt in [vhdl, verilog]:
        s = Simulator(tgt)
        s.run(inst, 20, debug = True)
    
test(unit)

 Module unit: Existing instance unit, rename to unit_2 
 Writing 'unit_2' to file /tmp/unit_2.vhdl 
 Creating library file /tmp/module_defs.vhdl 
WORK DIR of instance [Instance unit_2 I/F: [// ID: unit_0 to unit]] /tmp/myirl_unit_cdwo__bb/
==== COSIM stdout ====
DONE
simulation stopped @1ns

 Writing 'unit_2' to file /tmp/unit_2.v 
 Note: Changing library path prefix to /tmp/ 
 Creating library file /tmp/module_defs.v 
ICARUS FILES ['/tmp/unit_2.v']
==== COSIM stdout ====
DONE 
Stop Simulation



**Note** The `VHDL` target is by default bound to the VHDL-2008 standard. If VHDL93 is required for some constructs, create an instance of `targets.vhdl.VHDL93()` instead. This will however display a failure assertion upon `StopSimulation`.

**Note** For Icarus featured simulations, it is important to use `StopSimulation` with indefinitely stimulated (clocked) test benches.

Further reading: [Simulation](simulation.ipynb)